In [64]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, rigidity_modulus
from concrete_design.steel_reinforcement import rebar_df as rebar
from masonry_design.material_properties import NW_CMU, cmu_elastic_modulus
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Masonry Beam Design

### Material Properties

In [65]:
fm = 1500 #psi
fr = 267 #psi

Em = cmu_elastic_modulus(fm) #ksi
Es = 29000 #ksi

### Geometry

In [66]:
b = 9.625 #in
h = 4.25*12 #in
l = 46.5 #ft

#### Loading

In [67]:
RDL = 204 #plf
wall_weight = 181 * h/12 #plf
RLL = 240 #plf
RSL = 516 #plf

#support = "pinned-pinned"
support = "fixed-fixed"

w = load_combos.ASD(D=(RDL+wall_weight), Lr=RLL, S=RSL)
f"w = {round(w, 2)} plf"

'w = 1489.25 plf'

In [68]:
if support == "pinned-pinned":
    M = w * l**2 / 8 #ft-lbs
elif support == "fixed-fixed":
    M = w * l**2 / 12 #ft-lbs
    
f"M = {round(M/1000, 2)} kip-ft"

'M = 268.34 kip-ft'

### Reinforcement
#### Flexural Reinforcement

In [69]:
bar = "#6"
no_bars = 4
As = no_bars * rebar.loc[bar, "Area"]
d = h - 3 #in
rho = As / (b*d)
n = Es / Em
k = sqrt(2*rho*n + (rho*n)**2) - rho*n
j = 1 - k/3
f"As={As}, d={d}, rho={round(rho, 5)}, n={round(n, 3)}, j={round(j, 4)}, k={round(k, 4)}"

'As=1.76, d=48.0, rho=0.00381, n=21.481, j=0.8897, k=0.3309'

#### Shear Reinforcement

## ASD Moment Design

In [70]:
Fb = 0.45 * fm #psi
Fs = 32000 #psi

In [71]:
Mm = Fb * b * d**2 * (j*k/2) / 1000/12 #kip-ft
f"Mm = {round(Mm, 2)} kip-ft"

'Mm = 183.63 kip-ft'

In [72]:
Ms = As * Fs * j * d / 1000/12 #kip-ft
f"Ms = {round(Ms, 2)} kip-ft"

'Ms = 200.43 kip-ft'

In [73]:
Ma = min(Ms, Mm)
f"Ma = {round(Ma, 2)} kip-ft"

'Ma = 183.63 kip-ft'

## ASD Shear Design

In [74]:
#Fv = (Fvm + Fvs) * gamma_g

## Deflection

#### Deflections

In [75]:
Ig = b * h**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in^4")

Sg = b * h**2 / 6 #in^3
Mcr = fr * Sg / 12 #ft-lbs

Icr = (b*(k*d)**3)/3 + (n*rho*b*d)*(d - k*d)**2
print(f"Icr = {round(Icr, 2)} in^4")

Ieff = min(Ig * (Mcr / Ma)**3 + Icr * (1 - (Mcr / Ma)**3), Ig)
print(f"Ieff = {round(Ieff, 2)} in^4")

Ig = 106397.16 in^4
Icr = 51853.55 in^4
Ieff = 106397.16 in^4


In [76]:
if support == "pinned-pinned":
    defl = 5 * (w/12) * (l*12)**4 / (384 * (Em*1000) * Ieff)
elif support == "fixed-fixed":
    defl = (w/12) * (l*12)**4 / (384 * (Em*1000) * Ieff)
    
f"deflection = {round(defl, 4)} in"

'deflection = 0.2181 in'

In [77]:
defl_limit = l * 12 / 480
f"deflection limit = {round(defl_limit, 4)} in"

'deflection limit = 1.1625 in'

In [78]:
defl < defl_limit

True